In [1]:
import pandas as pd
import os
import glob
import numpy as np
import statistics as stats
import scipy
import src.mpra_tools.predicted_occupancy as po
import src.mpra_tools.fasta_utils as fu
import math
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import random
import requests, sys

## Change Ennsembl ID to Common Name.  The Cross Reference with known TF in mouse
### For the purpose of identifying immportant words/motifs in sentence construction

In [2]:
activity_df = pd.read_csv("Data/activity.csv", index_col=0)
retinopathy_df = pd.read_csv("Data/retinopathy.csv", index_col=0)
L = 164

In [3]:
motif_df = pd.read_parquet("Data/Motifs/summarized_all_motifs.parquet")
TFs = list(set(motif_df['motif']))

In [29]:
run = "WT3_run2"
rna_seq = pd.read_csv("Data/RNA/" + run + "/quant.sf", sep='\t', index_col=0)

expressed = rna_seq[rna_seq['TPM'] > 0.0]
expressed.index = expressed.index.map(lambda x: x.split('.')[0])
expressed

,Length,EffectiveLength,TPM,NumReads
Name,,,,
ENSMUST00000162897,4153,3870.979,0.663768,63.588
ENSMUST00000159265,2989,2724.613,0.635434,42.846
ENSMUST00000070533,3634,3341.441,1.010561,83.566
ENSMUST00000192857,480,281.415,0.143588,1.000
ENSMUST00000195335,2819,2578.917,0.031337,2.000
...,...,...,...,...
ENSMUST00000082418,1824,1593.169,1841.821022,72618.041
ENSMUST00000082419,519,315.231,2184.650206,17043.011
ENSMUST00000082420,69,13.000,37.299463,12.000


In [30]:
gene_info = pd.read_csv("Data/RNA/" + run + "/gene_info.csv", index_col=0)
gene_info['display_name'] = gene_info['display_name'].map(lambda x: x.split('-')[0].upper())
protein_info = gene_info[gene_info['biotype']=='protein_coding']
TF_info = protein_info[protein_info['display_name'].isin(TFs)].copy()
TF_info.head()

/tmp/ipykernel_3483716/427715528.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_info = pd.read_csv("Data/RNA/" + run + "/gene_info.csv", index_col=0)


,display_name,species,Parent,assembly_name,is_canonical,start,length,source,id,strand,biotype,version,object_type,db_type,end,logic_name,seq_region_name
ENSMUST00000027035,SOX17,mus_musculus,ENSMUSG00000025902,GRCm39,1,4561154,3127,ensembl_havana,ENSMUST00000027035,-1,protein_coding,10,Transcript,core,4566636,ensembl_havana_transcript_mus_musculus,1
ENSMUST00000186857,STAT1,mus_musculus,ENSMUSG00000026104,GRCm39,0,52158651,4241,havana,ENSMUST00000186857,1,protein_coding,7,Transcript,core,52200166,havana_mus_musculus,1
ENSMUST00000186574,STAT1,mus_musculus,ENSMUSG00000026104,GRCm39,0,52158599,2834,havana,ENSMUST00000186574,1,protein_coding,7,Transcript,core,52194571,havana_mus_musculus,1
ENSMUST00000070968,STAT1,mus_musculus,ENSMUSG00000026104,GRCm39,0,52158614,5151,ensembl_havana,ENSMUST00000070968,1,protein_coding,14,Transcript,core,52201024,ensembl_havana_transcript_mus_musculus,1
ENSMUST00000186057,STAT1,mus_musculus,ENSMUSG00000026104,GRCm39,1,52158783,3998,havana,ENSMUST00000186057,1,protein_coding,7,Transcript,core,52199889,havana_mus_musculus,1


In [31]:
TF_expression = expressed[expressed.index.isin(TF_info.index)].copy()
TF_expression['TF'] = TF_expression.index.map(lambda x: TF_info.loc[x, 'display_name'])
TF_expression.head()

,Length,EffectiveLength,TPM,NumReads,TF
Name,,,,,
ENSMUST00000027035,3127,2715.220,0.044646,3.000,SOX17
ENSMUST00000168302,2662,2447.935,0.148562,9.000,STAT4
ENSMUST00000186574,2834,2643.327,4.338706,283.822,STAT1
ENSMUST00000070968,5151,4896.225,0.412907,50.032,STAT1
ENSMUST00000186857,4241,4007.631,13.621207,1350.948,STAT1


In [32]:
TF_expression.to_parquet("Data/RNA/" + run + "/TF_expression.parquet")



### Get motif names by batched direct api calls to rest.ensembl.org

In [ ]:
# batch_size = 128
# ensembl_ids = [id.split('.')[0] for id in expressed.index.to_list()]
# batched_ids = []

# while len(ensembl_ids) >= batch_size:
#     batched_ids.append(ensembl_ids[:batch_size])
#     del ensembl_ids[:batch_size]
# batched_ids.append(ensembl_ids)

# server = "https://rest.ensembl.org/lookup/id"
# headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

# batched_jsons = [
#     '{ "ids" : ["' + '", "'.join(id_batch) + '" ] }' for id_batch in batched_ids
# ]

# gene_info = []
# counter = 0
# for batch in batched_jsons:
#     r = requests.post(server, headers=headers, data=batch)

#     if not r.ok:
#         r.raise_for_status()
#         sys.exit()
    
#     decoded = r.json()
    
#     gene_info.append(pd.DataFrame(decoded).T)
#     counter +=1
#     if counter % 50 ==0:
#         print(counter, '/', len(batched_jsons))
# print(counter, '/', len(batched_jsons))

# gene_info = pd.concat(gene_info)
# gene_info = gene_info.dropna()
# gene_info.to_csv("Data/RNA/" + run + "/gene_info.csv")
# gene_info